In [ ]:
### Set up 
!pip install torch torchvision gym

In [1]:
import math, random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

from ipywidgets import FloatProgress
from IPython.display import display

import pandas as pd
import seaborn as sns

import metadict_save as ms
import datetime

import warnings
warnings.filterwarnings('ignore')

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

plt.rcParams.update({'font.size': 12})
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    print("Using", torch.cuda.device_count() ,"GPUs.")
    gpu_ids = list(range(torch.cuda.device_count()))
else:
    print("Using CPU.")
    gpu_ids = []

Using CPU.


In [2]:
torch.manual_seed(0)
np.random.seed(0)
from modules import *

In [3]:
beta_start = 0.4
beta_frames = 1000
beta_by_frame = lambda frame_idx: min(1.0, beta_start + frame_idx * (1.0 - beta_start) / beta_frames)

epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

In [5]:
### Test agent on standard or noisy env 

In [11]:
# Test agent on standard or noisy env
# Resulting episode reward stored in XXX_val_rewards where XXX is standard or noisy
def test(val_env, noisyGame, eps, num_val_trials, current_model):
    rewards = []
    for i in range(num_val_trials):
        epsilon = 0
        episode_reward = 0
        state = val_env.reset()
        state = np.append(state, float(noisyGame))
        with torch.no_grad():
            while True:
                original_action = current_model.act(state, epsilon)

                if original_action != int(original_action):
                    original_action = original_action.numpy()[0]

                actual_action = original_action
                next_state, reward, done, _ = val_env.step(actual_action)
                next_state = np.append(next_state, float(noisyGame))

                if noisyGame:
                    reward += random.uniform(-1., 1.)

                state = next_state
                episode_reward += reward

                if done:
                    rewards.append(episode_reward)
                    break
    return np.mean(rewards)

In [12]:
### Train agent 

In [13]:
def train(env, val_env, variance, theta, exp, average_q_values, meg_norm, hardcoded, invert_actions = False, num_frames = 10000, num_val_trials = 10, batch_size = 32, gamma = 0.99, num_trials = 5, USE_CUDA = False, device = "", eps = 1.):

    num_trials = 1
    
    # Progress bar
    f = FloatProgress(min=0, max=num_frames)
    display(f)

    device = torch.device("cuda")

    """Args:"""
    losses = []
    all_rewards = []
    standard_val_rewards = []
    noisy_val_rewards = []
    states_count_ratios = []
    episode_reward = 0

    # Initialize state
    noisyGame = False
    state = env.reset()
    state = np.append(state, float(noisyGame)) # BACK IN
    meta_state = (state, float(noisyGame))

    # Initialize replay buffer, model, TD loss, and optimizers
    replay_buffer = PrioritizedBuffer(100000)
    # replay_buffer = PrioritizedBuffer(1000)
    current_model = DQN(env.observation_space.shape[0] + 1, env.action_space.n) # BACK IN
    target_model  = DQN(env.observation_space.shape[0] + 1, env.action_space.n) # BACK IN
    td_loss = TDLoss(average_q_values=average_q_values)
    optimizer = optim.Adam(current_model.parameters())
    
    # Multi GPU - Under Construction.
#     current_model = current_model.to(device)
#     target_model = target_model.to(device)

#     # Single GPU Code
    if USE_CUDA:
        current_model = nn.DataParallel(current_model, gpu_ids)
        target_model = nn.DataParallel(target_model, gpu_ids)
        current_model = current_model.cuda()
        target_model  = target_model.cuda()

    result_df = pd.DataFrame()
    power = theta
    var = variance
    all_standard_val_rewards = []
    all_proportions = []
    std_weights = []
    noisy_weights = []
    std_buffer_example_count = []
    noisy_buffer_example_count = []
    
    # TODO: restructure code to incorporate epsilon as experiment hyperparam
    # eps=.5

    for t in range(num_trials):
        print("trial number: {}".format(t))
        #print("is noisy game?", noisyGame)
        for frame_idx in range(1, num_frames + 1):
            epsilon = epsilon_by_frame(frame_idx)
            original_action = current_model.act(state, epsilon)

            # If in noisy environment, make action random with probability eps
            if noisyGame and random.uniform(0,1) < eps:
                if invert_actions:
                    actual_action = 1 - original_action # invert
                else:
                    actual_action = original_action
            else:
                actual_action = original_action

            next_state, reward, done, _ = env.step(actual_action)

            # If in noisy environment, make reward completely random
            if noisyGame:
                reward *= np.random.normal(0, var)

            next_state = np.append(next_state, float(noisyGame))
            meta_next_state = (next_state, float(noisyGame))
            replay_buffer.push(meta_state, original_action, reward, meta_next_state, done)

            meta_state = meta_next_state
            episode_reward += reward

            if done:
                noisyGame = 1-noisyGame
                state = env.reset()
                state = np.append(state, float(noisyGame))
                meta_state = (state, float(noisyGame))
                all_rewards.append(episode_reward)
                episode_reward = 0

            if len(replay_buffer) > batch_size:
                beta = beta_by_frame(frame_idx)
                # TODO: to compare with PER paper, shouldn't this be every 4 frames?
                loss = td_loss.compute_td_loss(current_model, target_model, beta, replay_buffer, optimizer)
                losses.append(loss.data.tolist())

            if frame_idx % 200 == 0:
                all_standard_val_rewards.append(test(val_env, False, eps, num_val_trials, current_model))
                all_proportions.append(float(replay_buffer.states_count[1]) / (float(replay_buffer.states_count[1])  + float(replay_buffer.states_count[0])))
                weight_dict = replay_buffer.get_average_weight_by_env()
                std_weights.append(weight_dict['std_avg'])
                noisy_weights.append(weight_dict['noisy_avg'])
                std_buffer_example_count.append(weight_dict['std_count'])
                noisy_buffer_example_count.append(weight_dict['noisy_count'])
                
                # Verified that standard experience replay weights noisy examples more heavily and oversamples
                # TODO: run complete experiments and check improvement with proposed replay technique
#                 print(weight_dict)
#                 print('Noisy to std weight', noisy_weights[-1]/std_weights[-1])
#                 print('Proportion of noisy selected (long run): ', all_proportions[-1])
                
            #         plot(frame_idx, all_rewards, losses, standard_val_rewards, noisy_val_rewards, states_count_ratios)

            if frame_idx % 1000 == 0:
                update_target(current_model, target_model)
                print("frame_idx", frame_idx)

            f.value += 1

    result_df['trial_num'] = list(range(int(num_frames / 200))) * num_trials # 50 = 10000 frames / 200 frams
    result_df['frame'] = result_df['trial_num'] * 200
    result_df['val_reward'] = all_standard_val_rewards
    result_df['proportion'] = all_proportions
    result_df['std_weights'] = std_weights
    result_df['noisy_weights'] = noisy_weights
    result_df['std_buffer_example_count'] = std_buffer_example_count
    result_df['noisy_buffer_example_count'] = noisy_buffer_example_count
    return result_df

In [14]:
### Run experiments 

In [17]:
def run_experiments(variances = np.logspace(0.1, 10, 4), thetas = np.logspace(0.1, 5, 4), exps = [False], meg_norms = [True, False], hardcodeds=[True, False], average_q_values=[False]):
    meta_dict = {}
    #pd.DataFrame(columns=['var', 'theta', 'exp', 'meg_norm', 'result_df'])
    for var in variances:
        for theta in thetas:
            for exp in exps:
                for avg_q in average_q_values:
                    for meg_norm in meg_norms:
                        for hardcoded in hardcodeds:
                            env_id = "CartPole-v0"
                            env, val_env = get_env(env_id)
                            result_df = train(env, val_env, var, theta, exp, avg_q, meg_norm, hardcoded, invert_actions = False, num_frames = 10000, num_val_trials = 10, batch_size = 32, num_trials = 5, USE_CUDA = USE_CUDA)
                            # TODO: make more descriptive
                            key = (var, theta, exp, avg_q, hardcoded)
                            meta_dict[key] = result_df
    return meta_dict

In [18]:
# meta_dict = run_experiments(variances = np.logspace(0.1, 10, 4), 
#                             thetas = np.logspace(0.1, 5, 4), 
#                             exps = [False], 
#                             meg_norms = [True, False], 
#                             hardcodeds=[True, False])

# TODO: restructure code for desired experiments
meta_dict = run_experiments(variances = [0], 
                            thetas = [1], 
                            exps = [False], 
                            meg_norms = [False], 
                            hardcodeds=[False],
                            average_q_values=[False, True])

FloatProgress(value=0.0, max=10000.0)

trial number: 0
frame_idx 1000
frame_idx 2000
frame_idx 3000
frame_idx 4000
frame_idx 5000
frame_idx 6000
frame_idx 7000
frame_idx 8000
frame_idx 9000
frame_idx 10000


FloatProgress(value=0.0, max=10000.0)

trial number: 0
frame_idx 1000
frame_idx 2000
frame_idx 3000
frame_idx 4000
frame_idx 5000
frame_idx 6000
frame_idx 7000
frame_idx 8000
frame_idx 9000
frame_idx 10000


In [ ]:
meta_dict

In [ ]:
# Save results
curr_dt = datetime.datetime.now()
save_dir = 'results/' + curr_dt.strftime('%m-%y-%d_%H:%M:%S.%f') + 'eps=.5var=0' + '/'
print(save_dir)

ms.save(meta_dict, save_dir)

In [ ]:
ms.load(save_dir)